**The Battle of Neighborhoods final notebook**

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import pickle 
import os
!conda install -c conda-forge folium=0.5.0
import folium
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import sklearn as skl
from sklearn.cluster import KMeans

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  10.14 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  23.03 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  25.20 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  34.45 MB/s


In [12]:
CLIENT_ID ='MK51CHNXA04S2LWHNEVGIAIC43GBF1CPZKPPDVNLY35XFGLJ'
CLIENT_SECRET = 'JHFP3TDXYXYBIABED0UBWA53WRGDTO5IYGY0GZTTZBW33QK0'
VERSION = '20190101' # Foursquare API version

In [13]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] # define the dataframe columns
neighborhoods = pd.DataFrame(columns=column_names)# create the dataframe
for i in range(len(neighborhoods_data)): #Iterate through each entry in the JSON file
    borough = neighborhoods_data[i]['properties']['borough'] #Boroughs first
    neighborhood_name = neighborhoods_data[i]['properties']['name'] #Followed by Neighborhood name
    neighborhood_lat = neighborhoods_data[i]['properties']['bbox'][1] #Now latitude
    neighborhood_lon = neighborhoods_data[i]['properties']['bbox'][2] #And longitude
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
mask = neighborhoods['Borough'] == 'Brooklyn' #Make a boolean mask for Manhattan Borough
df = neighborhoods.copy() #Make a copy to leave our original DF intact
df = df[mask] #Now use the mask to eliminate entries not in Manhattan
df.reset_index(inplace = True) #Now reset the index so we can use it
df.drop('index', axis = 1, inplace = True)
df.head() #And take a look at our work so far

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue ID'          
                            ]
    
    return(nearby_venues)

In [16]:
radius = 500
LIMIT = 20

brooklyn_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [17]:
brooklyn_venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa,4b895827f964a5206c2d32e3
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop,4ad09cf7f964a520bed820e3
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar,538ca7d5498ec684c6387a46
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot,4abe4e84f964a520958c20e3
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place,4ea17d24e5facb29c6d81d6f
5,Bay Ridge,40.625801,-74.030621,XIN,40.625082,-74.030494,Chinese Restaurant,5553702c498ecb85f99d8f37
6,Bay Ridge,40.625801,-74.030621,Brooklyn Market,40.626939,-74.029948,Grocery Store,5127e5c7e4b0447b9f67b41d
7,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.623348,-74.031082,Pizza Place,4a343f1ef964a520f69b1fe3
8,Bay Ridge,40.625801,-74.030621,Karam,40.622931,-74.028316,Middle Eastern Restaurant,4b6785d6f964a520aa532be3
9,Bay Ridge,40.625801,-74.030621,A.L.C. Italian Grocery,40.623051,-74.031224,Grocery Store,5089d4f7e4b0b2a614e157c5


In [18]:
len(brooklyn_venues)

1228

In [19]:
uniques = brooklyn_venues['Venue Category'].unique()
uniques

array(['Spa', 'Bagel Shop', 'Juice Bar', 'Breakfast Spot', 'Taco Place',
       'Chinese Restaurant', 'Grocery Store', 'Pizza Place',
       'Middle Eastern Restaurant', 'Greek Restaurant', 'Sports Bar',
       'Bookstore', 'Italian Restaurant', 'Bar', 'Caucasian Restaurant',
       'Hookah Bar', 'Cosmetics Shop', 'Bakery', 'Butcher',
       'Sushi Restaurant', 'Ice Cream Shop', 'Noodle House',
       'Shabu-Shabu Restaurant', 'Liquor Store', 'Hotpot Restaurant',
       'American Restaurant', 'Deli / Bodega', 'Donut Shop',
       'Mexican Restaurant', 'Latin American Restaurant', 'Bank',
       'Record Shop', 'Mobile Phone Shop', 'Gym', 'Yoga Studio',
       'Polish Restaurant', 'Laundry Service', 'Café', 'French Restaurant',
       'Fried Chicken Joint', 'Gymnastics Gym', 'Arts & Crafts Store',
       'Gastropub', 'Furniture / Home Store', 'Cocktail Bar', 'Lounge',
       'Pharmacy', 'Martial Arts Dojo', 'Gourmet Shop', 'Baseball Field',
       'Wine Shop', 'Automotive Shop', 'Music S

In [20]:
import re
uniqueList = uniques.tolist()
len(uniqueList)

223

In [21]:
targets = ['Restaurant', 'Joint', 'Place', 'Theater', 'Music', 'Club', 'club', 'Cocktail', 'cocktail', 'Roof', \
           'Art', 'Gallery', 'Café', 'Lounge', 'Pub', 'pub', 'Metro', 'Speakeasy', 'House'\
          'Hall', 'Office', 'Coworking', 'Cafe', 'Market', 'Arcade', 'Barcade', 'Auditorium'\
          'Nightlife', 'Party', 'Spa', 'Museum', 'Orchestra', 'Dance']

matching = [s for s in uniqueList if any(xs in s for xs in targets)]
len(matching)

81

In [22]:
mask = brooklyn_venues['Venue Category'].isin(matching)
filtered = brooklyn_venues[mask]
filtered.reset_index(inplace = True)
filtered.drop('index', axis = 1, inplace = True)
filtered.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa,4b895827f964a5206c2d32e3
1,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place,4ea17d24e5facb29c6d81d6f
2,Bay Ridge,40.625801,-74.030621,XIN,40.625082,-74.030494,Chinese Restaurant,5553702c498ecb85f99d8f37
3,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.623348,-74.031082,Pizza Place,4a343f1ef964a520f69b1fe3
4,Bay Ridge,40.625801,-74.030621,Karam,40.622931,-74.028316,Middle Eastern Restaurant,4b6785d6f964a520aa532be3


In [23]:
restaurants = ['Restaurant', 'Joint', 'Place']
matching = [s for s in uniqueList if any(xs in s for xs in restaurants)]
mealMask = filtered['Venue Category'].isin(matching)
nightMask = ~mealMask
eats = filtered[mealMask]
nights = filtered[nightMask]
nights.reset_index(inplace = True)
nights.head()

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa,4b895827f964a5206c2d32e3
1,6,Bay Ridge,40.625801,-74.030621,Mimi Nails,40.622571,-74.031477,Spa,4bd34524462cb7133601de07
2,10,Bay Ridge,40.625801,-74.030621,Skin Institute of New York,40.627033,-74.026492,Spa,4b951b8af964a520798f34e3
3,32,Greenpoint,40.730201,-73.954241,Early,40.732069,-73.954721,Café,54d43863498e653d2ab8343f
4,37,Greenpoint,40.730201,-73.954241,Brooklyn Craft Company,40.730357,-73.953139,Arts & Crafts Store,57f599cb498ef66f0428b339


In [26]:
def getDetails(name, VENUE_ID):
    
    nightVenues=[]
    for name, VENUE_ID in zip(name, VENUE_ID):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
                )
            
        # make the GET request
        results = requests.get(url).json()
        
        # return only relevant information for each nearby venue
        nightVenues.append(results)
        
    return(nightVenues)

    #for i in range(len(details))
    #popularTimes = pd.DataFrame([item for popularHours in popularHours for item in popularHours])
    #popularTimes.columns = ['timeframes',        
                            
night_venues = getDetails(name=nights['Venue'],
                                   VENUE_ID=nights['Venue ID'],
                                  )

Pilo Arts Day Spa and Salon
Mimi Nails
Skin Institute of New York
Early
Brooklyn Craft Company
Freckle
Cafe Riviera
The Brew Inn
Ramona
Living Room Cafe
Lounge Bar Cave
OMG MMA
Music Spirit On U
Mega Vision Center
A & Z Nails
Cafe Aziza 7
Jazz. Club
Dunkin'
F1 Lounge
Jewish Children's Museum
Brooklyn Children's Museum
Bakerie
Chocolatte Crown Heights
MTA Subway - Kingston Ave (3)
Soho Cafe & Grill
Argyle Yarn Shop
Brunswick
Bearded Lady
Madre Mezcaleria
Konditori
Ode to Babel
SummerStage - Betsy Head Park
Brownsville Rockaway Youthmarket
Silver Factory
Weylin
Bed-Vyne Cocktail
Brown Butter
Brooklyn Historical Society
Henry Public
The Long Island Bar
Anor Spa
Elsa
Brooklyn Social
Carroll Gardens Greenmarket
Smith Canteen
The Brooklyn Strategist
The Painted Pot
Relaxing On Smith
Brooklyn Botanica
Pioneer Works
Rita
The Bell House
King Killer Studios
Brooklyn Art Space
Battalion Studios
Lucey's Lounge
Fort Greene Park Greenmarket
Dick & Janes
Elan Salon & Day Spa
Kareemation Music Studio


In [34]:
def getHours(name, VENUE_ID):
    
    nightHours=[]
    for name, VENUE_ID in zip(name, VENUE_ID):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}/hours??&client_id={}&client_secret={}&v={}'.format(
            VENUE_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
                )
            
        # make the GET request
        nightResults = requests.get(url).json()
        
        # return only relevant information for each nearby venue
        nightHours.append(nightResults)
        
    return(nightHours)

    #for i in range(len(details))
    #popularTimes = pd.DataFrame([item for popularHours in popularHours for item in popularHours])
    #popularTimes.columns = ['timeframes',        
                            
night_hours = getHours(name=nights['Venue'],
                                   VENUE_ID=nights['Venue ID'],
                                  )

Pilo Arts Day Spa and Salon
Mimi Nails
Skin Institute of New York
Early
Brooklyn Craft Company
Freckle
Cafe Riviera
The Brew Inn
Ramona
Living Room Cafe
Lounge Bar Cave
OMG MMA
Music Spirit On U
Mega Vision Center
A & Z Nails
Cafe Aziza 7
Jazz. Club
Dunkin'
F1 Lounge
Jewish Children's Museum
Brooklyn Children's Museum
Bakerie
Chocolatte Crown Heights
MTA Subway - Kingston Ave (3)
Soho Cafe & Grill
Argyle Yarn Shop
Brunswick
Bearded Lady
Madre Mezcaleria
Konditori
Ode to Babel
SummerStage - Betsy Head Park
Brownsville Rockaway Youthmarket
Silver Factory
Weylin
Bed-Vyne Cocktail
Brown Butter
Brooklyn Historical Society
Henry Public
The Long Island Bar
Anor Spa
Elsa
Brooklyn Social
Carroll Gardens Greenmarket
Smith Canteen
The Brooklyn Strategist
The Painted Pot
Relaxing On Smith
Brooklyn Botanica
Pioneer Works
Rita
The Bell House
King Killer Studios
Brooklyn Art Space
Battalion Studios
Lucey's Lounge
Fort Greene Park Greenmarket
Dick & Janes
Elan Salon & Day Spa
Kareemation Music Studio


In [35]:
for i in range(len(night_hours)):
    print(i)
    try:
        nightInfo.loc[i, 'Hours'] = night_hours[i]['response']['hours']
    except:
        nightInfo.loc[i, 'Hours'] = 'N/A'
    
    try:
        nightInfo.loc[i, 'Popular'] = night_hours[i]['response']['popular']['timeframes']
    except:
        nightInfo.loc[i, 'Popular'] = 'N/A'

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [36]:
nightInfo.drop('Popular', axis = 1, inplace = True)

In [37]:
type(nights['Venue ID'][0])

str

In [57]:
info = nights.join(nightInfo, lsuffix='l', rsuffix='r', how='inner', sort=True)
info.drop(info.columns[[0,2,3,5,6,11]], axis=1, inplace=True)


In [58]:
info.head()

,Neighborhood,Venue,Venue Category,Venue ID,Likes,Tips
0,Bay Ridge,Pilo Arts Day Spa and Salon,Spa,4b895827f964a5206c2d32e3,9.0,20.0
1,Bay Ridge,Mimi Nails,Spa,4bd34524462cb7133601de07,11.0,4.0
2,Bay Ridge,Skin Institute of New York,Spa,4b951b8af964a520798f34e3,7.0,1.0
3,Greenpoint,Early,Café,54d43863498e653d2ab8343f,120.0,27.0
4,Greenpoint,Brooklyn Craft Company,Arts & Crafts Store,57f599cb498ef66f0428b339,7.0,3.0


In [59]:
info.describe()


,Likes,Tips
count,50.000000,50.000000
mean,101.180000,24.860000
std,151.006513,37.669893
min,0.000000,0.000000
25%,9.250000,4.000000
50%,25.000000,9.000000
75%,118.250000,25.750000
max,562.000000,185.000000


In [60]:
popularPlacesMask = info['Likes'] > 22.5
hotTipsMask = info['Tips'] > 8
combined = popularPlacesMask | hotTipsMask
popularPlaces = info[combined]
len(popularPlaces)
popularPlaces.head()

,Neighborhood,Venue,Venue Category,Venue ID,Likes,Tips
0,Bay Ridge,Pilo Arts Day Spa and Salon,Spa,4b895827f964a5206c2d32e3,9.0,20.0
3,Greenpoint,Early,Café,54d43863498e653d2ab8343f,120.0,27.0
5,Greenpoint,Freckle,Spa,4ee8b464754ab0f2605823e9,26.0,13.0
6,Greenpoint,Cafe Riviera,Café,4a6b3156f964a52025ce1fe3,29.0,23.0
7,Greenpoint,The Brew Inn,Gastropub,5285412911d2a3e51484ff56,123.0,26.0


In [61]:
popularCount = popularPlaces.groupby(['Neighborhood'], as_index = False).count()
popularCount

,Neighborhood,Venue,Venue Category,Venue ID,Likes,Tips
0,Bay Ridge,1,1,1,1,1
1,Bedford Stuyvesant,1,1,1,1,1
2,Brooklyn Heights,1,1,1,1,1
3,Carroll Gardens,5,5,5,5,5
4,Cobble Hill,3,3,3,3,3
5,Crown Heights,2,2,2,2,2
6,Gravesend,2,2,2,2,2
7,Greenpoint,5,5,5,5,5
8,Kensington,1,1,1,1,1
9,Prospect Heights,4,4,4,4,4


In [63]:
countMask = popularCount['Venue'] > 1
popNeighborhoods = popularCount[countMask]
neighList = popNeighborhoods['Neighborhood'].tolist()
neighList

['Carroll Gardens',
 'Cobble Hill',
 'Crown Heights',
 'Gravesend',
 'Greenpoint',
 'Prospect Heights',
 'Red Hook',
 'Windsor Terrace']

In [64]:
filtered.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa,4b895827f964a5206c2d32e3
1,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place,4ea17d24e5facb29c6d81d6f
2,Bay Ridge,40.625801,-74.030621,XIN,40.625082,-74.030494,Chinese Restaurant,5553702c498ecb85f99d8f37
3,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.623348,-74.031082,Pizza Place,4a343f1ef964a520f69b1fe3
4,Bay Ridge,40.625801,-74.030621,Karam,40.622931,-74.028316,Middle Eastern Restaurant,4b6785d6f964a520aa532be3


In [65]:
hoodMask = filtered['Neighborhood'].isin(neighList)
popVenues = filtered[hoodMask]
neighMask = df['Neighborhood'].isin(neighList)
popNeighborhoods = df[neighMask]

In [66]:
popNeighborhoods


,Borough,Neighborhood,Latitude,Longitude
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471
9,Brooklyn,Crown Heights,40.670829,-73.943291
12,Brooklyn,Windsor Terrace,40.656946,-73.980073
13,Brooklyn,Prospect Heights,40.676822,-73.964859
19,Brooklyn,Cobble Hill,40.687920,-73.998561
20,Brooklyn,Carroll Gardens,40.680540,-73.994654
21,Brooklyn,Red Hook,40.676253,-74.012759


In [67]:
radius = 500
LIMIT = 100

hot_venues = getNearbyVenues(names=popNeighborhoods['Neighborhood'],
                                   latitudes=popNeighborhoods['Latitude'],
                                   longitudes=popNeighborhoods['Longitude']
                                  )
len(hot_venues)

Greenpoint
Gravesend
Crown Heights
Windsor Terrace
Prospect Heights
Cobble Hill
Carroll Gardens
Red Hook


511

In [68]:
uniques = hot_venues['Venue Category'].unique()
hotUniqueList = uniques.tolist()
hotUniqueList

['Yoga Studio',
 'Polish Restaurant',
 'Laundry Service',
 'Mexican Restaurant',
 'Café',
 'Bar',
 'Pizza Place',
 'Grocery Store',
 'French Restaurant',
 'Fried Chicken Joint',
 'Gymnastics Gym',
 'Arts & Crafts Store',
 'Spa',
 'Italian Restaurant',
 'Record Shop',
 'Gastropub',
 'Furniture / Home Store',
 'Cocktail Bar',
 'Deli / Bodega',
 'Coffee Shop',
 'Sushi Restaurant',
 'Bookstore',
 'Beer Store',
 'Restaurant',
 'Liquor Store',
 'Vegetarian / Vegan Restaurant',
 'Vape Store',
 'Boutique',
 'Sandwich Place',
 'Wine Shop',
 'Salon / Barbershop',
 'New American Restaurant',
 'Vietnamese Restaurant',
 'Bakery',
 'Whisky Bar',
 'Beer Bar',
 'Fruit & Vegetable Store',
 'Thai Restaurant',
 'Tea Room',
 'Nightclub',
 'Ice Cream Shop',
 'Bagel Shop',
 'Optical Shop',
 'Coworking Space',
 'American Restaurant',
 'Donut Shop',
 'Juice Bar',
 'Wine Bar',
 'Dance Studio',
 'Antique Shop',
 'Playground',
 'Falafel Restaurant',
 'Ramen Restaurant',
 'Nail Salon',
 'Burger Joint',
 'Rental C

In [69]:
newtargets = ['Restaurant', 'Joint', 'Place', 'Theater', 'Music', 'Club', 'club', 'Cocktail', 'cocktail', 'Roof', \
           'Art', 'Gallery', 'Café', 'Lounge', 'Pub', 'pub', 'Metro', 'Speakeasy', 'House'\
          'Hall', 'Office', 'Coworking', 'Cafe', 'Market', 'Arcade', 'Barcade', 'Auditorium'\
          'Nightlife', 'Party', 'Spa', 'Museum', 'Orchestra', 'Dance', 'Record', 'Boutique', 'Pie', 'Beer', 'Video'\
          'Event', 'Movie', 'Concert', 'Hostel', 'Hotel', 'Laser']

hotMatches = [s for s in hotUniqueList if any(xs in s for xs in newtargets)]
len(hotMatches)

71

In [70]:
mask = hot_venues['Venue Category'].isin(matching)
hotFilter = hot_venues[mask]
hotFilter.reset_index(inplace = True)
hotFilter.drop('index', axis = 1, inplace = True)
hotFilter.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Greenpoint,40.730201,-73.954241,Karczma,40.730102,-73.955092,Polish Restaurant,4a916658f964a520151a20e3
1,Greenpoint,40.730201,-73.954241,Oxomoco,40.729981,-73.955460,Mexican Restaurant,5ae8ed8eb3c961002b3360da
2,Greenpoint,40.730201,-73.954241,Friducha,40.731512,-73.954281,Mexican Restaurant,5a2b5dbdc0af5755f9b41a7a
3,Greenpoint,40.730201,-73.954241,Fornino,40.728951,-73.953862,Pizza Place,519bdb24abd81726b7a90ede
4,Greenpoint,40.730201,-73.954241,Citroën,40.731229,-73.954564,French Restaurant,5bf358689fca56002c251a87


In [120]:
hotRestaurants = ['Restaurant', 'Joint', 'Place']
hotRestaurantsMatch = [s for s in hotUniqueList if any(xs in s for xs in hotRestaurants)]
print(len(hotRestaurantsMatch))
hotRestMask = hot_venues['Venue Category'].isin(hotRestaurantsMatch)
hotNightMask = ~hotRestMask
hotEats = hotFilter[hotRestMask]
hotNights = hot_venues[hotNightMask]
hotNights.reset_index(inplace = True)
hotNights.drop('index', axis = 1, inplace = True)
hotNights.head()

40


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Greenpoint,40.730201,-73.954241,goodyoga,40.730010,-73.956167,Yoga Studio,4a9a86dcf964a520cc3120e3
1,Greenpoint,40.730201,-73.954241,Sunshine Laundry & Pinball Emporium,40.729318,-73.953564,Laundry Service,4c018f2e716bc9b6c319bc55
2,Greenpoint,40.730201,-73.954241,Early,40.732069,-73.954721,Café,54d43863498e653d2ab8343f
3,Greenpoint,40.730201,-73.954241,Black Rabbit,40.730057,-73.956588,Bar,46869096f964a52048481fe3
4,Greenpoint,40.730201,-73.954241,The Garden,40.730760,-73.954502,Grocery Store,49c2b82ef964a5201e561fe3


In [83]:
len(hotNights), len(hotEats)

(356, 148)

In [84]:
hotNightCounts = hotNights.groupby(['Neighborhood'], as_index = False).count()
hotEatCounts = hotEats.groupby(['Neighborhood'], as_index = False).count()
hotEatCounts.sort_values(by = ['Neighborhood'], axis = 0, ascending = True, inplace = True)
hotNightCounts.sort_values(by = ['Neighborhood'], axis = 0, ascending = True, inplace = True)

In [85]:
hotNightCounts


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Carroll Gardens,71,71,71,71,71,71,71
1,Cobble Hill,65,65,65,65,65,65,65
2,Crown Heights,15,15,15,15,15,15,15
3,Gravesend,23,23,23,23,23,23,23
4,Greenpoint,68,68,68,68,68,68,68
5,Prospect Heights,53,53,53,53,53,53,53
6,Red Hook,39,39,39,39,39,39,39
7,Windsor Terrace,22,22,22,22,22,22,22


In [91]:
popNeighborhoods.sort_values(by = ['Neighborhood'], axis = 0, ascending = True, inplace = True)
popNeighborhoods.reset_index(inplace = True)
popNeighborhoods.drop('index', axis = 1, inplace = True)
popNeighborhoods


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,level_0,Borough,Neighborhood,Latitude,Longitude
0,0,Brooklyn,Carroll Gardens,40.680540,-73.994654
1,1,Brooklyn,Cobble Hill,40.687920,-73.998561
2,2,Brooklyn,Crown Heights,40.670829,-73.943291
3,3,Brooklyn,Gravesend,40.595260,-73.973471
4,4,Brooklyn,Greenpoint,40.730201,-73.954241
5,5,Brooklyn,Prospect Heights,40.676822,-73.964859
6,6,Brooklyn,Red Hook,40.676253,-74.012759
7,7,Brooklyn,Windsor Terrace,40.656946,-73.980073


In [92]:
popNeighborhoods['Night Counts'] = hotNightCounts['Venue']
popNeighborhoods['Restaurant Counts'] = hotEatCounts['Venue']
popNeighborhoods

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,level_0,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts
0,0,Brooklyn,Carroll Gardens,40.680540,-73.994654,71,27
1,1,Brooklyn,Cobble Hill,40.687920,-73.998561,65,31
2,2,Brooklyn,Crown Heights,40.670829,-73.943291,15,4
3,3,Brooklyn,Gravesend,40.595260,-73.973471,23,7
4,4,Brooklyn,Greenpoint,40.730201,-73.954241,68,32
5,5,Brooklyn,Prospect Heights,40.676822,-73.964859,53,30
6,6,Brooklyn,Red Hook,40.676253,-74.012759,39,11
7,7,Brooklyn,Windsor Terrace,40.656946,-73.980073,22,6


In [96]:
hotNeighCounts = hot_venues.groupby(['Neighborhood'], as_index = False).count()

popNeighborhoods['Total Counts'] = hotNeighCounts['Venue']
popNeighborhoods

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,level_0,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts
0,0,Brooklyn,Carroll Gardens,40.680540,-73.994654,71,27,100
1,1,Brooklyn,Cobble Hill,40.687920,-73.998561,65,31,100
2,2,Brooklyn,Crown Heights,40.670829,-73.943291,15,4,19
3,3,Brooklyn,Gravesend,40.595260,-73.973471,23,7,30
4,4,Brooklyn,Greenpoint,40.730201,-73.954241,68,32,100
5,5,Brooklyn,Prospect Heights,40.676822,-73.964859,53,30,84
6,6,Brooklyn,Red Hook,40.676253,-74.012759,39,11,50
7,7,Brooklyn,Windsor Terrace,40.656946,-73.980073,22,6,28


In [97]:
popNeighborhoods['Night Frequency'] = popNeighborhoods['Night Counts'] / popNeighborhoods['Total Counts']
popNeighborhoods['Eats Frequency'] = popNeighborhoods['Restaurant Counts'] / popNeighborhoods['Total Counts']
popNeighborhoods

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,level_0,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts,Night Frequency,Eats Frequency
0,0,Brooklyn,Carroll Gardens,40.680540,-73.994654,71,27,100,0.710000,0.270000
1,1,Brooklyn,Cobble Hill,40.687920,-73.998561,65,31,100,0.650000,0.310000
2,2,Brooklyn,Crown Heights,40.670829,-73.943291,15,4,19,0.789474,0.210526
3,3,Brooklyn,Gravesend,40.595260,-73.973471,23,7,30,0.766667,0.233333
4,4,Brooklyn,Greenpoint,40.730201,-73.954241,68,32,100,0.680000,0.320000
5,5,Brooklyn,Prospect Heights,40.676822,-73.964859,53,30,84,0.630952,0.357143
6,6,Brooklyn,Red Hook,40.676253,-74.012759,39,11,50,0.780000,0.220000
7,7,Brooklyn,Windsor Terrace,40.656946,-73.980073,22,6,28,0.785714,0.214286


In [98]:
restaurants = hotEats['Venue Category'].unique()
restList = restaurants.tolist()
restList

['Polish Restaurant',
 'Mexican Restaurant',
 'Pizza Place',
 'French Restaurant',
 'Fried Chicken Joint',
 'Italian Restaurant',
 'Sushi Restaurant',
 'Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Sandwich Place',
 'New American Restaurant',
 'Vietnamese Restaurant',
 'Thai Restaurant',
 'American Restaurant',
 'Falafel Restaurant',
 'Ramen Restaurant',
 'Burger Joint',
 'Indian Restaurant',
 'Chinese Restaurant',
 'Middle Eastern Restaurant',
 'Caribbean Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Cajun / Creole Restaurant',
 'Southern / Soul Food Restaurant',
 'Greek Restaurant',
 'Spanish Restaurant',
 'Japanese Restaurant',
 'Seafood Restaurant',
 'Mediterranean Restaurant',
 'Cuban Restaurant',
 'Asian Restaurant',
 'BBQ Joint']

In [99]:
chineseTargets = ['Chinese', 'Hong Kong', 'Taiwan', 'Noodle', 'Ramen', 'Dumpling', 'Asian', 'Japanese']

chineseMatch = [s for s in restList if any(xs in s for xs in chineseTargets)]

In [100]:
chineseMask = hotEats['Venue Category'].isin(chineseMatch)
chineseFilter = hotEats[chineseMask]
chineseFilter.reset_index(inplace = True)
chineseFilter.drop('index', axis = 1, inplace = True)
chineseFilter.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Greenpoint,40.730201,-73.954241,Ramen Mafia,40.732713,-73.958005,Ramen Restaurant,5b5a55eb2463d80044aa5775
1,Gravesend,40.595260,-73.973471,Choy Le Chinese Restaraunt,40.597537,-73.968531,Chinese Restaurant,4db9e9370437ffd20796f4af
2,Gravesend,40.595260,-73.973471,Zheng China Kitchen,40.592010,-73.977453,Chinese Restaurant,4c1825f3834e2d7fbc842880
3,Windsor Terrace,40.656946,-73.980073,East Wind Snack Shop,40.660297,-73.980169,Chinese Restaurant,54e39b14498e9c646b42958e
4,Prospect Heights,40.676822,-73.964859,Chuko,40.680732,-73.967616,Ramen Restaurant,4e306b3c18383aa77b01b3d2


In [101]:
chiNeighborCounts = chineseFilter.groupby(['Neighborhood'], as_index = False).count()
chiNeighborCounts

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Carroll Gardens,1,1,1,1,1,1,1
1,Cobble Hill,3,3,3,3,3,3,3
2,Gravesend,2,2,2,2,2,2,2
3,Greenpoint,1,1,1,1,1,1,1
4,Prospect Heights,1,1,1,1,1,1,1
5,Windsor Terrace,1,1,1,1,1,1,1


In [103]:
for i in range(len(chiNeighborCounts)):
    place = chiNeighborCounts.loc[i, 'Neighborhood']
    row = popNeighborhoods.loc[popNeighborhoods['Neighborhood']==place].index
    popNeighborhoods.loc[row, 'Chinese Restaurants'] = chiNeighborCounts.loc[i, 'Venue']

popNeighborhoods.fillna(0.0)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,level_0,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts,Night Frequency,Eats Frequency,Chinese Restaurants
0,0,Brooklyn,Carroll Gardens,40.680540,-73.994654,71,27,100,0.710000,0.270000,1.0
1,1,Brooklyn,Cobble Hill,40.687920,-73.998561,65,31,100,0.650000,0.310000,3.0
2,2,Brooklyn,Crown Heights,40.670829,-73.943291,15,4,19,0.789474,0.210526,0.0
3,3,Brooklyn,Gravesend,40.595260,-73.973471,23,7,30,0.766667,0.233333,2.0
4,4,Brooklyn,Greenpoint,40.730201,-73.954241,68,32,100,0.680000,0.320000,1.0
5,5,Brooklyn,Prospect Heights,40.676822,-73.964859,53,30,84,0.630952,0.357143,1.0
6,6,Brooklyn,Red Hook,40.676253,-74.012759,39,11,50,0.780000,0.220000,0.0
7,7,Brooklyn,Windsor Terrace,40.656946,-73.980073,22,6,28,0.785714,0.214286,1.0


In [104]:
popNeighborhoods['Chinese Frequency'] = popNeighborhoods['Chinese Restaurants'] / popNeighborhoods['Total Counts']
popNeighborhoods.fillna(0.0)


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,level_0,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts,Night Frequency,Eats Frequency,Chinese Restaurants,Chinese Frequency
0,0,Brooklyn,Carroll Gardens,40.680540,-73.994654,71,27,100,0.710000,0.270000,1.0,0.010000
1,1,Brooklyn,Cobble Hill,40.687920,-73.998561,65,31,100,0.650000,0.310000,3.0,0.030000
2,2,Brooklyn,Crown Heights,40.670829,-73.943291,15,4,19,0.789474,0.210526,0.0,0.000000
3,3,Brooklyn,Gravesend,40.595260,-73.973471,23,7,30,0.766667,0.233333,2.0,0.066667
4,4,Brooklyn,Greenpoint,40.730201,-73.954241,68,32,100,0.680000,0.320000,1.0,0.010000
5,5,Brooklyn,Prospect Heights,40.676822,-73.964859,53,30,84,0.630952,0.357143,1.0,0.011905
6,6,Brooklyn,Red Hook,40.676253,-74.012759,39,11,50,0.780000,0.220000,0.0,0.000000
7,7,Brooklyn,Windsor Terrace,40.656946,-73.980073,22,6,28,0.785714,0.214286,1.0,0.035714


In [106]:
clusterDF = popNeighborhoods.copy()
clusterDF.drop('Borough', axis = 1, inplace = True)
clusterDF.drop('Neighborhood', axis = 1, inplace = True)
clusterDF.drop('Latitude', axis = 1, inplace = True)
clusterDF.drop('Longitude', axis = 1, inplace = True)
clusterDF.drop('Restaurant Counts', axis = 1, inplace = True)
clusterDF.drop('Night Counts', axis = 1, inplace = True)

In [107]:
clusterDF.fillna(0.0, inplace = True)
clusterDF['Chinese Frequency'].astype(float)

0    0.010000
1    0.030000
2    0.000000
3    0.066667
4    0.010000
5    0.011905
6    0.000000
7    0.035714
Name: Chinese Frequency, dtype: float64

In [108]:
type(clusterDF.loc[1, 'Chinese Frequency'])

numpy.float64

In [115]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clusterDF)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:8]

array([0, 0, 1, 1, 0, 3, 2, 1], dtype=int32)

In [116]:
popNeighborhoods['Cluster'] = kmeans.labels_
popNeighborhoods


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,level_0,Borough,Neighborhood,Latitude,Longitude,Night Counts,Restaurant Counts,Total Counts,Night Frequency,Eats Frequency,Chinese Restaurants,Chinese Frequency,Cluster
0,0,Brooklyn,Carroll Gardens,40.680540,-73.994654,71,27,100,0.710000,0.270000,1.0,0.010000,0
1,1,Brooklyn,Cobble Hill,40.687920,-73.998561,65,31,100,0.650000,0.310000,3.0,0.030000,0
2,2,Brooklyn,Crown Heights,40.670829,-73.943291,15,4,19,0.789474,0.210526,NaN,NaN,1
3,3,Brooklyn,Gravesend,40.595260,-73.973471,23,7,30,0.766667,0.233333,2.0,0.066667,1
4,4,Brooklyn,Greenpoint,40.730201,-73.954241,68,32,100,0.680000,0.320000,1.0,0.010000,0
5,5,Brooklyn,Prospect Heights,40.676822,-73.964859,53,30,84,0.630952,0.357143,1.0,0.011905,3
6,6,Brooklyn,Red Hook,40.676253,-74.012759,39,11,50,0.780000,0.220000,NaN,NaN,2
7,7,Brooklyn,Windsor Terrace,40.656946,-73.980073,22,6,28,0.785714,0.214286,1.0,0.035714,1


**Cluster 2 seems to have a good number of nightlife venues, zero numbers of Chinese Restaurants, and a balanced number of other Restaurants. This cluster is a good starting point to look for a new location for our restaurant**

In [117]:
# create map
latitude = 40.685683
longitude = -73.983748

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(popNeighborhoods['Latitude'], popNeighborhoods['Longitude'], popNeighborhoods['Neighborhood'], popNeighborhoods['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters